# LSTM implementation for TORCS driver

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data

## LSTM Network

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, batch_size):
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        
        super(RNN, self).__init__()        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.out = nn.Linear(hidden_size, output_size)        
        self.hidden = self.init_hidden()
        
    def init_hidden(self, x=None):
        if x == None:
            return (Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)))
        else:
            return (Variable(x[0].data),Variable(x[1].data))
        
    def forward(self, x):
        lstm_out, self.hidden_out = self.lstm(x, self.hidden)
        output = self.out(lstm_out.view(len(x), -1))
        self.hidden = self.init_hidden(self.hidden_out)
        return output

In [10]:
INPUT_SIZE = 22
HIDDEN_SIZE = 100
NUM_LAYERS = 2
BATCH_SIZE = 100
NUM_EPOCHS = 15
LEARNING_RATE = 0.001

rnn = RNN(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, 3, BATCH_SIZE)

optimizer = torch.optim.Adam(rnn.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()

## Train network

In [11]:
filenames = ['train_data/aalborg.csv', 'train_data/alpine-1.csv', 'train_data/f-speedway.csv']
X_train = None
y_train = None

for file in filenames:
    dataset = pd.read_csv(file)
    
    X = dataset.iloc[:, 3:].values
    y = dataset.iloc[:, :3].values
    
    X_train = X if X_train is None else np.concatenate((X_train, X))
    y_train = y if y_train is None else np.concatenate((y_train, y))
        
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
imputer = imputer.fit(X_train)
X_train = imputer.transform(X_train)

X_train = torch.from_numpy(X_train)
X_train = X_train.float()
y_train = torch.from_numpy(y_train)
y_train = y_train.float()

ds = torch.utils.data.TensorDataset(X_train, y_train)

train_loader = torch.utils.data.DataLoader(dataset=ds,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)

In [12]:
for epoch in range(NUM_EPOCHS):
    for i, (X, y) in enumerate(train_loader):
        if (len(X) != BATCH_SIZE):
            continue
        
        params = Variable(X.view(-1, 1, INPUT_SIZE))
        command = Variable(y)

        optimizer.zero_grad()
        prediction = rnn(params)
        loss = criterion(prediction, command)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, NUM_EPOCHS, i+1, len(X_train)//100, loss.data[0]))

print('Training done')

Epoch [1/15], Step [100/240], Loss: 0.0426
Epoch [1/15], Step [200/240], Loss: 0.0358
Epoch [2/15], Step [100/240], Loss: 0.0307
Epoch [2/15], Step [200/240], Loss: 0.0506
Epoch [3/15], Step [100/240], Loss: 0.0474
Epoch [3/15], Step [200/240], Loss: 0.0451
Epoch [4/15], Step [100/240], Loss: 0.0199
Epoch [4/15], Step [200/240], Loss: 0.0327
Epoch [5/15], Step [100/240], Loss: 0.0386
Epoch [5/15], Step [200/240], Loss: 0.0232
Epoch [6/15], Step [100/240], Loss: 0.0350
Epoch [6/15], Step [200/240], Loss: 0.0238
Epoch [7/15], Step [100/240], Loss: 0.0223
Epoch [7/15], Step [200/240], Loss: 0.0286
Epoch [8/15], Step [100/240], Loss: 0.0202
Epoch [8/15], Step [200/240], Loss: 0.0170
Epoch [9/15], Step [100/240], Loss: 0.0428
Epoch [9/15], Step [200/240], Loss: 0.0307
Epoch [10/15], Step [100/240], Loss: 0.0188
Epoch [10/15], Step [200/240], Loss: 0.0269
Epoch [11/15], Step [100/240], Loss: 0.0155
Epoch [11/15], Step [200/240], Loss: 0.0249
Epoch [12/15], Step [100/240], Loss: 0.0239
Epoch 

In [13]:
torch.save(rnn.state_dict(), 'rnn_params.pt')

In [14]:
torch.save(rnn, 'whole_net.pt')

/usr/local/miniconda3/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
